### Perimeter Institute Nov 21st
### 2018 computational physics course
# Exact Diagonalization
 
Guifre Vidal  

# IJulia nb3: Low energy spectrum and quantum criticality

In [ ]:
using PyPlot
using LinearAlgebra
using Arpack

In [ ]:
# function buildH takes as input N (number of spins) and theta (magnetic field angle) and outputs H (Ising Hamiltonian)

function buildH(N::Int64,theta::Float64)::Array{Float64,2} 
    X = [0. 1; 1 0]
    Z = [1. 0; 0 -1]
    E = diagm(0=>ones(2))
    XX = kron(X,X)
    HXX = XX
    HZ = kron(Z,E) + kron(E,Z)
    for n = 3:N
        HXX = kron(HXX,E)+kron(diagm(0=>ones(2^(n-2))), XX)
        HZ =  kron(HZ,E) + kron(diagm(0=>ones(2^(n-1))),Z)
    end
    HXX = HXX + kron(X,kron(diagm(0=>ones(2^(N-2))),X))
    H = -cos(theta)*HXX - sin(theta)*HZ
    return H
end

buildH(3,pi/4) # test for N=3 and critical theta = pi/4


### functions eigen(H) vs eigs(H, nev=2)


In [ ]:
N = 8
H = buildH(N,pi/2)
D,U = eigen(H)
d,u = eigs(H, nev=2, which=:SR) # :LR largest real, :SR smallest real, :LM largest magnitude, :SM smallest magnitude 
display([D[1] d[1]])
Psi1 = U[:,1]
Psi2 = u[:,1]
display(abs(Psi1'*Psi2))

### Excercise (1/2):  check the scaling of computational time with $N$, for eigen and eigs with nev$=1$ and nev$=8$

In [ ]:
Nmin = 4 
Nmax = 10  # Nmax ~ 10 recomended, 11 OK, but not much larger!!!
time_full = zeros(Nmax-Nmin + 1)
theta = pi/3
n_repeat = 1;

In [ ]:
# full
for N in Nmin:Nmax
    H = buildH(N,theta)

    t1 = time()
    for n=1:n_repeat 
        D,U = eigen(H)
    end
    t2 = time()
    time_full[N-Nmin+1] = t2-t1
    display([size(H) t2-t1])
    sleep(0.5)
end

In [ ]:
semilogy([i for i in Nmin:Nmax+1], [(3e-10)*2^(3i) for i in Nmin:Nmax+1], color = "r")
semilogy([i for i in Nmin:Nmax+1], [(3e-8)*2^(2i) for i in Nmin:Nmax+1],  color = "r")

semilogy([i for i in Nmin:Nmax], time_full, marker = "s", color = "k")


In [ ]:
N=10
Nstates = 12
H = buildH(N,pi/4)
D,U = eigs(H, nev=Nstates, which=:SR) # :LR largest real, :SR smallest real, :LM largest magnitude, :SM smallest magnitude 
plot(collect(1:Nstates), sort(D), marker = "o", markersize = 10, color = "b")

### Conformal data (scaling dimensions and conformal spins) of the Ising CFT
### Let us plot the 12 lowest scaling dimensions (and corresponding conformal spins)

In [ ]:
# Plot exact scaling dimensions (Delta) and conformal spins (spin) in each parity sector (parity) 
Ndim = 12
Delta = [0 1/8 1 1+1/8 1+1/8 2 2 2 2 2+1/8 2+1/8 2+1/8]
spin = [0 0 0 1 -1 2 1 -1 -2 2 0 -2]
parity = [0 1 0 1 1 0 0 0 0 1 1 1]

figure("conformal_data_pbc",figsize=(10,4))

subplot(121) # Create the 1st axis of a 2x2 arrax of axes
grid("on") # Create a grid on the axis
title("Even Parity, p=0")
ax = gca()
ax[:set_xlim]([-3,3])
ax[:set_ylim]([-0.2,2.8])
xlabel("conformal spin")
ylabel("scaling dimension")
annotate("identity", xy=[0.2;0])
annotate(L"$\mathbb{I}$", xy=[-0.3;0.2])
annotate("energy density", xy=[0.2;1])
annotate(L"$\epsilon$", xy=[-0.3;1.2])

subplot(122) # Create the 1st axis of a 2x2 arrax of axes
grid("on") # Create a grid on the axis
title("Odd Parity, p=1")
ax = gca()
ax[:set_xlim]([-3,3])
ax[:set_ylim]([-0.2,2.8])
xlabel("conformal spin")
ylabel("scaling dimension")
annotate("spin", xy=[0.2;1/8])
annotate(L"$\sigma$", xy=[-0.3;0.3])

for i=1:Ndim
    if parity[i] == 0
        subplot(121)
        plot(spin[i],Delta[i], marker = "s", markersize = 10, color = "y")
    elseif parity[i] == 1
        subplot(122)
        plot(spin[i],Delta[i], marker = "s", markersize = 10, color = "y")
    end
end 

### Exercise (2/2): obtain & plot numerical estimates for 12 scaling dimensions and conformal spins ($N=10$ spins)
### hint: In any CFT, the lowest energy state with momentum $k 2 \times 2\pi/N$ (or spin $s = 2$) corresponds to a CFT operator called stress tensor, which has  $(\Delta_T,s_T) = (2,2)$. 
### Find the stress tensor energy eigenstate $|T\rangle$ and use its energy to estimate the scaling dimensions $\Delta_i$ of the rest of energy eigenstates according to $\Delta_i = 2 (e_i-e_1)/(e_T - e_1)$. 

### Here are buildT and buildP for the transaltion and parity operators (from last Friday)

In [ ]:
# function buildT takes as input N (number of spins) and outputs T (translatio operator)
function buildT(N::Int64)::Array{Float64,2} 
    SWAP = [1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 1]
    E = diagm(0=>ones(2))
    T = copy(SWAP)
    for n=3:N
        SWAPn = kron(diagm(0=>ones(2^(n-2))), SWAP)
        T =  SWAPn*kron(T,E)
    end
    return T
end

buildT(3) # test for N=3  

In [ ]:
# function buildP takes as input N (number of spins) and outputs P (parity operator)
function buildP(N::Int64)::Array{Float64,2} 
    Z = [1 0; 0 -1]
    P = copy(Z)
    for i=2:N
        P = kron(P,Z)
    end 
    return P
end

buildP(3) # test for N=3  